## Define Dynamic Parameters

In [5]:
file_no = 0
my_seed = 64

## Import Packages

In [6]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import ntpath

tf.random.set_seed(my_seed)
np.random.seed(my_seed)

## Import CSVs

In [7]:
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '\\..\\..\\Datasets\\processed\\regression_10d'
print(path)

csvfiles = glob.glob(path + "/*.csv")
dfs = []

for file in csvfiles:
    df = pd.read_csv(file)
    df.name = (ntpath.basename(file)).split(".",1)[0]
    dfs.append(df)

df = dfs[file_no]
print("Stock: ")
print(df.name)
print(my_seed)

C:\Users\Dell\Documents\B.Sc IT (Hons.) Computing and Business\FYP - Luke Bezzina\Code\mlpLearning\..\..\Datasets\processed\regression_10d
Stock: 
AAL
64


# Splitting dataset in dependent and independent variables

In [8]:
x = np.array(df.iloc[:, 1:-1].values, dtype=np.float)
#x = np.array(df.iloc[:, 0:-1].values)
y = np.array(df.iloc[:, -1:].values)


# Splitting dataset (training + testing)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False, random_state=32)

print("Train - Independent Variables")
print(x_train)
print("Test - Independent Variables")
print(x_test)
# print("Train - Dependent Variables")
# print(y_train)
# print("Test - Dependent Variables")
# print(y_test)

# trg_cols_to_std = x_train[:, 10:12]
# test_cols_to_std = x_test[:, 10:12]
#
# scaler = StandardScaler()
# scaler.fit(trg_cols_to_std)
# trg_cols_to_std = scaler.transform(trg_cols_to_std)
# test_cols_to_std = scaler.transform(test_cols_to_std)
#
# x_train[:, 10:12] = trg_cols_to_std
# x_test[:, 10:12] = test_cols_to_std

Train - Independent Variables
[['2017-01-19' 0.4000000000000057 -0.4099999999999966 ...
  -0.030125666973886182 0.8916972580422119 47.64]
 ['2017-01-20' -0.8100000000000023 -0.490000000000002 ...
  -0.054502054164215684 0.8686656433864525 47.26]
 ['2017-01-23' 0.3200000000000003 1.1899999999999975 ...
  -0.03558202675732302 0.7284977693857406 48.0]
 ...
 ['2020-08-06' 0.41000000000000014 0.030000000000001137 ...
  -0.016212034220245232 0.41545276506481466 12.56]
 ['2020-08-07' -0.37999999999999895 -0.37999999999999895 ...
  0.052660918865283464 0.6189022539949259 13.04]
 ['2020-08-10' 0.0 0.37999999999999895 ... 0.08843578471566407
  0.7369559009872978 13.03]]
Test - Independent Variables
[['2020-08-11' 0.37999999999999895 0.009999999999999787 ...
  0.10958565887892988 0.9695798058953166 14.0]
 ['2020-08-12' -0.3699999999999992 -0.5899999999999999 ...
  0.1561962851234767 1.081752744392174 13.73]
 ['2020-08-13' -0.22000000000000064 -0.28000000000000114 ...
  0.1466658764314185 1.101746

NameError: name 'dgdf' is not defined

# Building ANN with MLP

In [ ]:
ann = tf.keras.models.Sequential()
# hidden layers
ann.add(tf.keras.layers.Dense(units=7, activation='relu'))
# ann.add(tf.keras.layers.Dense(units=5, activation='relu'))
# output layer
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))

optimizer = tf.keras.optimizers.Adam()
ann.compile(optimizer = optimizer, loss = 'mean_squared_error')

ann.fit(x_train, y_train, batch_size = 32, epochs = 200)

weights = ann.get_weights()

# Predicting Test Set Results

In [ ]:
y_pred = ann.predict(x_test)
# boolean result of whether predicted probability is larger than 0.5
# result > 0.5 = 1, result <= 0.5 = 0
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))


# Obtaining Confusion Matrix and Accuracy Score for predictions

In [ ]:
from sklearn.metrics import mean_absolute_error


print()
print("Mean Absolute Error: ")
mean_absolute_error(y_test, y_pred)

# Classifying results

In [ ]:
prediction_arr = []
actuals_arr = []
neutral_percentage = 0.002

for i in range(1, len(y_pred)):
    if y_pred[i] - y_test[i-1] > (y_pred[i] * neutral_percentage):
            prediction_arr.append([1,0,0])
    elif y_pred[i] - y_test[i-1]  < - (y_pred[i] * neutral_percentage):
            prediction_arr.append([0,0,1])
    else:
            prediction_arr.append([0,1,0])

for i in range(1, len(y_test)):
    if y_test[i] - y_test[i-1] > (y_test[i] * neutral_percentage):
            actuals_arr.append([1,0,0])
    elif y_test[i] - y_test[i-1]  < - (y_test[i] * neutral_percentage):
            actuals_arr.append([0,0,1])
    else:
            actuals_arr.append([0,1,0])

actuals_arr = np.array(actuals_arr)
prediction_arr = np.array(prediction_arr)



# Results

In [ ]:
%%capture cap

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

# Confusion Matrix: Columns = 0, 1 (Predicted Label)
# Confusion Matrix: Rows = 0, 1 (Actual Label)
cm = multilabel_confusion_matrix(np.argmax(actuals_arr, axis=1), np.argmax(prediction_arr, axis=1))
print(cm)
print()

print("Accuracy Score: ")
print(accuracy_score(actuals_arr, prediction_arr))

# Graphs

In [ ]:
%matplotlib inline

with open(ROOT_DIR+'\\regression_results\\'+df.name + '_' + str(my_seed) +'.txt', 'w') as f:
    f.write(cap.stdout)
    
test_nos = []
for i, val in enumerate(y_test):
    test_nos.append(i)
plt.plot(test_nos, y_test)
plt.plot(test_nos, y_pred, color='green')